In [1]:
from utils.process_data import Process

In [2]:
import os

DATA_PATH = './data/cc_2018/'
files = [f for f in os.listdir(DATA_PATH)]
example = Process(files, DATA_PATH, 'pdf-output-2.pkl')
OVERWRITE = True

In [3]:
process_val = {'stopwords':'english',
               'stemmer':'yes',
               'ngrams':'bigrams'}

temp_ = example.remove_lines(OVERWRITE, example, **process_val)

file exists, loading data...
loaded...


In [5]:
params = {'vectorizor': 'tfidf',
          'max_df': .95,
          'min_df': 20,
          'max_features':None 
         }

tfidf = example.vectorize(**params)

tfidf

loading data...
loaded...
applying tfidf


<48451x12090 sparse matrix of type '<class 'numpy.float64'>'
	with 1890275 stored elements in Compressed Sparse Row format>

In [6]:
import operator
def rank_terms(A, terms):
    # get the sums over each column
    sums = A.sum(axis=0)
    # map weights to the terms
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    # rank the terms by their weight over all documents
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

In [194]:
snippets,raw_docs = [],[]

for f in files:
    doc = example._strip(f,DATA_PATH)
    raw_docs.append(doc)
    for d in doc:
        text = l.strip()
        snippets.append(text[0:min(len(text),100)])
        
print(f'got text from {len(raw_docs)} documents')

AttributeError: 'list' object has no attribute 'strip'

In [58]:
#[l.strip()[0:min(len(l),100)] for l in [example._strip(f,DATA_PATH) for f in files][0]]

In [59]:
from sklearn.externals import joblib

(tfidf,terms) = joblib.load("./data/tfidf-output.pkl")
print("Loaded %d X %d document-term matrix" % (tfidf.shape[0], tfidf.shape[1]))

Loaded 48451 X 12090 document-term matrix


In [60]:
ranking = rank_terms(tfidf, terms)

In [61]:
for i, pair in enumerate( ranking[0:20] ):
    print("%02d. %s (%.2f)" % ( i+1, pair[0], pair[1]))

01. credit (1622.99)
02. account (1483.24)
03. fee (1370.47)
04. card (1327.65)
05. payment (1192.24)
06. agreement (1089.62)
07. rate (1085.20)
08. balanc (1068.93)
09. charg (976.28)
10. purchas (961.21)
11. may (939.98)
12. amount (932.46)
13. interest (921.72)
14. apr (873.79)
15. transact (803.54)
16. use (798.58)
17. credit_card (759.85)
18. bill (735.20)
19. advanc (712.73)
20. pay (711.12)


In [62]:
joblib.dump((tfidf,terms,snippets), "articles-tfidf.pkl")

['articles-tfidf.pkl']

In [63]:
from sklearn.externals import joblib
(tfidf,terms,snippets) = joblib.load("articles-tfidf.pkl" )
print("Loaded %d X %d document-term matrix" % (tfidf.shape[0], tfidf.shape[1]))

Loaded 48451 X 12090 document-term matrix


In [64]:
kmin, kmax = 4, 15

In [65]:
from sklearn import decomposition
topic_models = []
# try each value of k
for k in range(kmin,kmax+1):
    print("Applying NMF for k=%d ..." % k )
    # run NMF
    model = decomposition.NMF(init="nndsvd", n_components=k) 
    W = model.fit_transform(tfidf)
    H = model.components_    
    # store for later
    topic_models.append((k,W,H))

Applying NMF for k=4 ...
Applying NMF for k=5 ...
Applying NMF for k=6 ...
Applying NMF for k=7 ...
Applying NMF for k=8 ...
Applying NMF for k=9 ...
Applying NMF for k=10 ...
Applying NMF for k=11 ...
Applying NMF for k=12 ...
Applying NMF for k=13 ...
Applying NMF for k=14 ...
Applying NMF for k=15 ...


In [121]:
import pandas as pd

df_ = pd.concat([temp_[k] for k in temp_.keys()],axis=0).reset_index().drop(['index'],axis=1)

In [147]:
txt = "hello, my name is Peter, I am 26 years old"

x = txt.split(", ")

print(x) 

['hello', 'my name is Peter', 'I am 26 years old']


In [150]:
df_[0].rows

AttributeError: 'Series' object has no attribute 'rows'

In [168]:
#the algorithm is splitting the words into individual letters I need to make it into a list
import itertools

l = [df_[0].iloc[x].split(", ") for x in range(652)]

#l = list(itertools.chain.from_iterable(l)); 
l

[['credit_card',
  'card_applic',
  'applic_disclosur',
  'credit',
  'card',
  'applic',
  'disclosur'],
 ['ibmsecu_ibm',
  'ibm_southeast',
  'southeast_employe',
  'employe_credit',
  'credit_union',
  'union_box',
  'box_boca',
  'boca_raton',
  'raton_www',
  'www_ibmsecu',
  'ibmsecu_org',
  'org_serviceplus',
  'serviceplus_ibmsecu',
  'ibmsecu_org',
  'ibmsecu',
  'ibm',
  'southeast',
  'employe',
  'credit',
  'union',
  'box',
  'boca',
  'raton',
  'www',
  'ibmsecu',
  'org',
  'serviceplus',
  'ibmsecu',
  'org'],
 ['credit_card',
  'card_applic',
  'applic_disclosur',
  'credit',
  'card',
  'applic',
  'disclosur'],
 ['import_credit',
  'credit_card',
  'card_disclosur',
  'disclosur_follow',
  'follow_disclosur',
  'disclosur_repres',
  'repres_import',
  'import_detail',
  'detail_concern',
  'concern_credit',
  'credit_card',
  'card_inform',
  'inform_cost',
  'cost_card',
  'card_accur',
  'accur_contact',
  'contact_toll',
  'toll_free',
  'free_address',
  'addre

In [169]:
import gensim

w2v_model = gensim.models.Word2Vec(l, size=1, min_count=20, sg=1)

In [172]:
len(w2v_model.wv.vocab)

374

In [188]:
from gensim.models import FastText

f2txt_model = FastText(l, min_count=1)

In [189]:
print("Model has %d terms" % len(f2txt_model.wv.vocab))

Model has 12129 terms


In [192]:
f2txt_model.save("./data/f2t-model.bin")

In [173]:
print("Model has %d terms" % len(w2v_model.wv.vocab))

Model has 374 terms


In [174]:
w2v_model.save("./data/w2v-model-2.bin")

In [190]:
def calculate_coherence( w2v_model, term_rankings ):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations( term_rankings[topic_index], 2 ):
            pair_scores.append( w2v_model.similarity(pair[0], pair[1]) )
        # get the mean for all pairs in this topic
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    # get the mean score across all topics
    return overall_coherence / len(term_rankings)

In [176]:
import numpy as np
def get_descriptor( all_terms, H, topic_index, top ):
    # reverse sort the values to sort the indices
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    # now get the terms corresponding to the top-ranked indices
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( all_terms[term_index] )
    return top_terms


In [191]:
from itertools import combinations
k_values = []
coherences = []
for (k,W,H) in topic_models:
    # Get all of the topic descriptors - the term_rankings, based on top 10 terms
    term_rankings = []
    for topic_index in range(k):
        term_rankings.append( get_descriptor( terms, H, topic_index, 10 ) )
    # Now calculate the coherence based on our Word2vec model
    k_values.append(k)
    coherences.append( calculate_coherence( f2txt_model, term_rankings ) )
    print("K=%02d: Coherence=%.4f" % ( k, coherences[-1] ) )

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys


K=04: Coherence=0.9942
K=05: Coherence=0.9952
K=06: Coherence=0.9971
K=07: Coherence=0.9963
K=08: Coherence=0.9954
K=09: Coherence=0.9965
K=10: Coherence=0.9969
K=11: Coherence=0.9958
K=12: Coherence=0.9962
K=13: Coherence=0.9958
K=14: Coherence=0.9963
K=15: Coherence=0.9965
